In [1]:
#type:ignore
import pandas as pd
import numpy as np
import re 
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer

In [2]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [3]:
# View the English stopwords that will be unnecessary for our analysis

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [4]:
try:
    news_df = pd.read_csv('IFND.csv', encoding='utf-8')
except UnicodeDecodeError:
    try:
        news_df = pd.read_csv('IFND.csv', encoding='latin-1')
    except UnicodeDecodeError:
        try:
            news_df = pd.read_csv('IFND.csv', encoding='iso-8859-1')
        except UnicodeDecodeError:
            news_df = pd.read_csv('IFND.csv', encoding='cp1252')


In [39]:
# df = news_df.drop(columns=['Unnamed: 0', 'title'])
# df

In [5]:
# View the dataframe's first few entries

news_df.head()

,id,Statement,Image,Web,Category,Date,Label
0,2,"WHO praises India's Aarogya Setu app, says it ...",https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,COVID-19,Oct-20,TRUE
1,3,"In Delhi, Deputy US Secretary of State Stephen...",https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,VIOLENCE,Oct-20,TRUE
2,4,LAC tensions: China's strategy behind delibera...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,TERROR,Oct-20,TRUE
3,5,India has signed 250 documents on Space cooper...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,COVID-19,Oct-20,TRUE
4,6,Tamil Nadu chief minister's mother passes away...,https://cdn.dnaindia.com/sites/default/files/s...,DNAINDIA,ELECTION,Oct-20,TRUE


In [6]:
# View number of null values in each column

news_df.isna().sum()

id               0
Statement        0
Image            0
Web              0
Category         0
Date         11321
Label            0
dtype: int64

In [7]:
# Fill our Null values with an empty space

news_df = news_df.fillna('')

In [8]:
# Separate the Target variable from the dataset

X = news_df.drop(columns=['Image','Date','id','Label','Web','Category'], axis=1)
Y = news_df['Label']

In [9]:
X

,Statement
0,"WHO praises India's Aarogya Setu app, says it ..."
1,"In Delhi, Deputy US Secretary of State Stephen..."
2,LAC tensions: China's strategy behind delibera...
3,India has signed 250 documents on Space cooper...
4,Tamil Nadu chief minister's mother passes away...
...,...
56709,Fact Check: This is not Bruce Lee playing ping...
56710,Fact Check: Did Japan construct this bridge in...
56711,Fact Check: Viral video of Mexico earthquake i...
56712,Fact Check: Ballet performance by Chinese coup...


In [10]:
Y

0        TRUE
1        TRUE
2        TRUE
3        TRUE
4        TRUE
         ... 
56709    Fake
56710    Fake
56711    Fake
56712    Fake
56713    Fake
Name: Label, Length: 56714, dtype: object

In [11]:
# Stemming process
p_stemming = PorterStemmer()

In [13]:
# Function to reduce the word to its original

def clean_text(text):
    text_cleaning_re = r"@\S+|https?://\S+|http?://\S|[^A-Za-z0-9]+"
    text = re.sub(text_cleaning_re, ' ', str(text).lower()).strip()
    
    tokens = text.split()
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if w not in stop_words]
    
    return " ".join(tokens)
    

In [14]:
X['stemmed_statement'] = X['Statement'].apply(clean_text)

In [15]:
X['stemmed_statement'] = X['stemmed_statement'].fillna('')
# Vectorizing using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(X['stemmed_statement'])

In [16]:
# Check the shape and unique values of the target labels (Y) to ensure data consistency
print("Shape of X:", X.shape)
print("Shape of Y:", Y.shape)
print("Unique values in Y:", Y.unique())

Shape of X: (56714, 2)
Shape of Y: (56714,)
Unique values in Y: ['TRUE' 'Fake']


In [17]:
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(X_tfidf, f)

In [18]:
try:
    # Attempt to split the data into train and test sets
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)
    
    # Check the shapes of the train and test sets
    print("Data split successfully!")
    print("Shape of X_train:", X_train.shape)
    print("Shape of X_test:", X_test.shape)
    print("Shape of Y_train:", Y_train.shape)
    print("Shape of Y_test:", Y_test.shape)
    
except Exception as e:
    # Print the error message if an error occurs during data splitting
    print("Error occurred during data splitting:", e)

Data split successfully!
Shape of X_train: (45371, 2)
Shape of X_test: (11343, 2)
Shape of Y_train: (45371,)
Shape of Y_test: (11343,)


In [19]:
# Fit and transform the training set, transform the test set
tfidf_train = tfidf_vectorizer.fit_transform(X_train['stemmed_statement'])
tfidf_test = tfidf_vectorizer.transform(X_test['stemmed_statement'])

In [20]:
# Initialize a DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()

In [21]:
# Train the model
decision_tree.fit(tfidf_train, Y_train)

with open('decision_tree_model.pkl', 'wb') as f:
    pickle.dump(decision_tree, f)

In [23]:
# Predict on the test set and calculate accuracy
y_pred = decision_tree.predict(tfidf_test)
accuracy = accuracy_score(Y_test, y_pred)
print(f'Decision Tree Accuracy: {round(accuracy * 100, 2)}%')


Decision Tree Accuracy: 91.92%


In [24]:
def predict_label_dt(input_text):
    input_tfidf = tfidf_vectorizer.transform([input_text])
    prediction = decision_tree.predict(input_tfidf)
    return prediction[0]

In [27]:

input = 'Narendra Modi is India Prime Minister'
print(predict_label_dt(input))

TRUE


In [28]:
logistic_regression = LogisticRegression()

In [29]:
logistic_regression.fit(tfidf_train, Y_train)

with open('logistic_regression_model.pkl', 'wb') as f:
    pickle.dump(logistic_regression, f)

In [30]:
# Predict on the test set and calculate accuracy
y_pred = logistic_regression.predict(tfidf_test)
accuracy = accuracy_score(Y_test, y_pred)
print(f'Logistic Regression Accuracy: {round(accuracy * 100, 2)}%')

Logistic Regression Accuracy: 93.48%


In [31]:
def predict_label_logistic(input_text):
    input_tfidf = tfidf_vectorizer.transform([input_text])
    prediction = logistic_regression.predict(input_tfidf)
    return prediction[0]

In [32]:
input = 'Narendra Modi is India Prime Minister'
print(predict_label_logistic(input))

TRUE


In [33]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

In [35]:
knn.fit(tfidf_train, Y_train)

with open('knn_model.pkl', 'wb') as f:
    pickle.dump(knn, f)

In [36]:
# Predict on the test set and calculate accuracy
y_pred = knn.predict(tfidf_test)
accuracy = accuracy_score(Y_test, y_pred)
print(f'KNN Accuracy: {round(accuracy * 100, 2)}%')

KNN Accuracy: 87.91%


In [37]:
def predict_label_knn(input_text):
    input_tfidf = tfidf_vectorizer.transform([input_text])
    prediction = knn.predict(input_tfidf)
    return prediction[0]

In [38]:
input = 'Narendra Modi is India Prime Minister'
print(predict_label_knn(input))

Fake
